In [39]:
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class Board:
    queens: List[Tuple[int, int]]
    done: bool
    
    def __str__(self):
        return str(self.done) + ", " + str(self.queens)
    
    def __repr__(self):
        return self.__str__()

class Solution:
    # given a List of queen positions (row, column), check if a new queen can be
    # placed at (row, col) without attacking any other queen
    def isValid(self, board: Board, row: int, col: int) -> bool:
        for queen in board.queens:
            if queen[0] == row or queen[1] == col or abs(queen[0] - row) == abs(queen[1] - col):
                return False
        return True

    # boards is a boards
    def addAllQueens(self, boards: List[Board], n:int) -> List[Board]:
        new_boards: List[Board] = []

        if len(boards) == 0:
            #print("no boards, setting up all initial boards")
            for i in range(n):
                for j in range(n):
                    new_boards.append(Board([(i, j)], False))
        else:
            #print("adding queens to existing boards")
            for board in boards:
                # if board is done, just add it to the new_boards
                if board.done:
                    new_boards.append(board)
                # otherwise, add all possible queens to the board
                else:
                    done = True
                    for i in range(n):
                        for j in range(n):
                            if self.isValid(board, i, j) and len(board.queens) < n:
                                new_boards.append(Board(board.queens + [(i, j)], False))
                                done = False
                    # if we couldn't add any queens, mark the board as done and add it to new_boards
                    if done and len(board.queens) == n:
                        new_boards.append(Board(board.queens, True))
        
        # sort the boards consistently so that we can filter out duplicates
        for board in new_boards:
            board.queens.sort()
        #print("new_boards:", new_boards)
        
        # filter out duplicates by converting each board to a tuple and adding to a set
        board_set = set()
        for board in new_boards:
            #print("  board:", board)
            tuple_board = tuple(item for pair in board.queens for item in pair) + (board.done,)
            board_set.add(tuple_board)
        #print("board_set:", board_set)
        
        # convert the tuples back to Boards
        new_boards: List[Board] = []
        for board in board_set:
            new_boards.append(Board([(board[2*i], board[2*i+1]) for i in range(len(board)//2)], board[-1]))
        #print("new_boards filtered:", new_boards)

        return new_boards
    
    def checkDone(self, boards: List[Board]) -> bool:
        for board in boards:
            if not board.done:
                return False
        return True

    def solveNQueens(self, n: int) -> List[List[str]]:
        boards: List[Board] = []
        boards = self.addAllQueens(boards, n)
        while not self.checkDone(boards):
            #print()
            boards = self.addAllQueens(boards, n)

        print("num final boards:", len(boards))

        return None        

In [46]:
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class Board:
    queens: List[Tuple[int, int]]
    
    def printBoard(self) -> List[str]:
        output = []
        for i in range(len(self.queens)):
            output.append("."*self.queens[i][1] + "Q" + "."*(len(self.queens) - self.queens[i][1] - 1))
        return output

    def __str__(self):
        return str(self.queens)
    
    def __repr__(self):
        return self.__str__()

# so, in the previous implementation, I had originally designed it under the misunderstanding that we wanted all 
# possible deadlocked n x n boards, not all possible deadlocked boards with n queens. So in the previous one, the
# original implementation was made to support placing queens at any possible location. For the n-queens case
# we can simplify this by noting that in each row and column there must be exactly one queen. So we can just
# start at the top row, add some queens and then go row by row adding queens to the board provided they
# can't attack any other queens. This gives an upper bound of n! (actually considerably lower) possible boards.
class Solution:
    # given a List of queen positions (row, column), check if a new queen can be
    # placed at (row, col) without attacking any other queen
    def isValid(self, board: Board, row: int, col: int) -> bool:
        for queen in board.queens:
            if queen[0] == row or queen[1] == col or abs(queen[0] - row) == abs(queen[1] - col):
                return False
        return True

    # either add all possible queens on the first row, or for each existing board state, add all possible
    # new board states with a queen in the next row.
    def addAllQueens(self, boards: List[Board], n:int, row:int) -> List[Board]:
        new_boards: List[Board] = []

        # if we are at row 0, we need to set up all possible initial boards
        if row == 0:
            #print("no boards, setting up all initial boards")
            for col in range(n):
                new_boards.append(Board([(row, col)]))
        else:
            for board in boards:
                for col in range(n):
                    if self.isValid(board, row, col):
                        new_boards.append(Board(board.queens + [(row, col)]))

        return new_boards

    def solveNQueens(self, n: int) -> List[List[str]]:
        boards: List[Board] = []
        boards = self.addAllQueens(boards, n, 0)
        for row in range(1, n):
            # print("row", row)
            boards = self.addAllQueens(boards, n, row)

        # print("num final boards:", len(boards))

        result = []
        for board in boards:
            result.append(board.printBoard())

        return result       

In [47]:
# test 1
n = 6
boards = Solution().solveNQueens(n)
print(boards)

[['.Q....', '...Q..', '.....Q', 'Q.....', '..Q...', '....Q.'], ['..Q...', '.....Q', '.Q....', '....Q.', 'Q.....', '...Q..'], ['...Q..', 'Q.....', '....Q.', '.Q....', '.....Q', '..Q...'], ['....Q.', '..Q...', 'Q.....', '.....Q', '...Q..', '.Q....']]
